## YOLO v1
- YOLO Paper : You Only Look Once:Unified, Real-Time Object Detection
- http://pjreddie.com/yolo/
- 이미지 전체에 대한 하나의 신경망(end to end)
- 단 한 번의 계산만으로 Bounding box와 클래스 확률(class probability)을 예측
- 1-stage Detector로 객체검출을 하나의 회귀문제(Single regression problem)로 봄

![YOLO](img/YOLO.PNG)

<hr>

## ■ YOLO의 장점과 단점
- 장점
    - (1) 단일 신경망 구조로 구성이 단순하고 매우 빠름
        - 모델의 학습과 최적화에 큰 이점을 줌. 
        - R-CNN과 같은 복잡한 모델일수록 Stage 별 개별학습이라 최적화가 힘듬
    - **(2) YOLO는 주변 정보까지 학습하여 이미지 전체를 처리하기 때문에 Background error가 작다.**
        - 이미지 전체를 본다 -> 즉 배경정보도 처리하고, R-CNN은 배경에 반점이나 노이즈가 있으면 그것을 물체로 인식
    - **(3) 물체의 일반적인 부분을 학습하기 떄문에 보다 일반화에 강건**
        - Object의 일반적인 부분 학습
    - (4) YOLO는 훈련과 테스트 단계에서 이미지 전체를 보게 됨.
        - 클래스의 모양 정보뿐 아니라 주변 정보도 학습하여 처리

- 단점
    - 작은 Obejct에 대해 상대적으로 낮은 정확도(MAP)를 보인다.


<hr>

## YOLO Netowork
- (1) YOLO는 입력 이미지를 S x S grid 로 나눈다.
- (2) 각각의 grid cell은 B개의 Bounding box를 가지며 각각은 그 Bounding Box에 대한 confidence score를 예측
- (3) Grid Cell 내에서의 Bounding Box 계산
    - 즉 0~1의 값을 가짐
        - ex1) 셀 중앙시 (x, y) = (0.5, 0.5)
        - ex2) 이미지 전체의 너비와 높이 (w, h)
- (4) 각각의 Grid Cell은 클래스별 Conditional class probabilities(C)를 예측
    - C = Pr(Class | Object) = 해당 클래스에 대한 조건부 확률 예측 (0~1)
- (5) 실제 Bounding Box와 예측 Bounding Box를 통한 Confidence 추출
     
![yolo_image](img/yolo_image.png)

## IOU (Intersection over union)
- **객체의 실제 bounding box와 예측 bounding box의 합집합 면적 대비 교집합 면적의 비율**
- (실제 bounding box와 예측 bounding box의 교집합) / (실제 bounding box와 예측 bounding box의 합집합)
- Pr(Object)=0 : 그리드 셀에 아무 객체가 없음. Confidence score
- Pr(Object)=1 : 그리드 셀에 어떤 객체가 확실히 있음
- **Confidence score와 IOU가 같다면 가장 이상적인 Score**

<hr>


## Network Design

![yolo_network2](img/yolo_network2.jpg)

- GoogleNet 모델 기반 (No 인셉션 모듈, 1x1 reduction layer 만 사용)
- 24 Conv layers & 2 Fully Connected layer 
- 마지막계층은 선형활성화함수, 나머지 계층은 leaky ReLU
- 구조상 문제 해결을 위한 3가지 개선안
    - localization loss와 classification loss중 localiztaion loss의 가중치 증가
    - 객체가 없는 그리드 셀의 confidence loss보다 객체가 존재하는 그리드 셀의 confidence loss의 가중치 증가
    - 과적합 막기 위한 dropout, data augmentatition 적용
- **98개의 class specific confidence score에 대해 각 20개의 클래스를 기준으로 non-maximum suppression을 진행**

- 최종 Output 
    - **클래스 확률(Class probabilities)**
        - **classification loss**
    - **Bounding box 위치정보(coordinates)**
        - **localization loss**
        - **너비 (width), 높이(height), bounding box 중심좌표(x,y)**
    

<hr>

## YOLO Training
- ImageNet 데이터셋으로 YOLO 앞단 20개의 컨볼루션 계층 Pre-train 사전훈련
- Pre-train 후 4conv 계층과 2fc 계층 추가.
- Bounding Box의 width와 Height 값은 normalize

## YOLO Loss Function

![yolo_loss_function](img/yolo_loss_function.jpg)

- YOLO의 loss는 SSE(sum-squared error)를 기반 / 최종 아웃풋의 SSE(sum-squared error)를 최적화(optimize)
- Object가 있는 경우, 없는 경우보다 큰 loss를 줌 (감마 coord=5, 감마 noobj=0.5) 
- 첫번쨰줄이 위치에 대한 loss / 두번째줄이 크기에 대한 loss
- 큰 물체에 대해서는 루트를 씌워서 영향을 조금 줄임

<hr>

## YOLO Inference
- **YOLO는 한 이미지당 98개의 Bounding box를 예측**하고, **Bounding box마다 클래스 확률을 구함**
- 단순 신경망 계산만 필요하기 때문에 계산이 매우 빠름

## YOLO NMS
- **Grid 디자인 시 하나의 객체를 여러 그리드 셀이 동시에 검출하는 경우 발생가능**
    - ex) 객체의 크기가 너무 큰 경우
    - ex2) 객체가 그리드 셀 경계에 인접해 있는 경우
- 해당 경우를 다중검출(multiple detections) 문제라고 함
- **비 최대 억제 (non-maximal suppression) 를 통한 개선** 
    - Non Maximum Suppression 으로 Leave only one Bounding Box
    - 바운딩 박스가 많을 때 최종 하나만 남기게 되는 것
    - (1) 동일한 클래스에 대해 confidence 정렬
    - (2) 가장 confidence 가 높은 bounding box를 기준으로 IOU가 Threshold를 넘는 Bounding box 제거
    - (3) 제거되지 않는 box들 중 (2)에서 선정된 박스 제외후 (1)~(2) 과정 반복
    - (4) 위 과정 반복시 중복된 box들은 모두 제거되고 거리가 떨어진 유의미한 Box들만 살아남음

## Limitation of YOLO
- **각각의 grid cell이 하나의 클래스만을 예측하므로 작은 Object 여러개가 겹칠시 예측 불가**
- 훈련단계에서 학습하지 못한 종횡비를 테스트 단계에서 마주치면 고전
- 큰 Bounding box와 작은 Bounding box의 loss에 동일 가중치를 둔다는 점
    - 크기가 큰 Bounding box는 위치가 조금 달라져도 성능에 영향을 주기 힘듬
    - 크기가 작은 Bounding box는 위치가 조금 달라져도 성능에 큰 영향 줄 수 있음
- 몇단계의 Layer를 거쳐 나온 Feature map을 대상으로 Bounding Box를 예측하여 Localization이 다소 부정확해 질 수 있음

<hr>

## 참고문헌
- https://transformer.tistory.com/entry/You-Only-Look-Once-Unified-Real-Time-Object-Detection-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0

- https://curt-park.github.io/2017-03-26/yolo/
- https://bkshin.tistory.com/entry/%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-YOLOYou-Only-Look-Once